In [1]:
import numpy as np
from numpy.linalg import norm
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
%matplotlib inline

In [2]:
# Sphere settings
RADIUS = 15
LINES = 12
RESOLUTION = 120

# Subspace settings
CUBES_PER_SIDE = 6

# Axes settings
AXES_LEN = 15

# Problem settings
SCALE = 1 / 2
MELBOURNE_LAT = np.deg2rad(-37.8140000)
MELBOURNE_LON = np.deg2rad(144.9633200)
SYDNEY_LAT = np.deg2rad(-33.8678500)
SYDNEY_LON = np.deg2rad(151.2073200)

In [3]:
def gen_points(points, transf_matrix):
    points = transf_matrix @ points
    line_params = {'mode': 'lines', 'showlegend': False, 'line': go.Line(color='#999', width=2)}
    return go.Scatter3d(x=points[0,:], y=points[1,:], z=points[2,:], **line_params)

def subspace(transf_matrix):
    cps_half = CUBES_PER_SIDE // 2
    traces = []
    values = np.arange(-cps_half, +cps_half+1).reshape(1, CUBES_PER_SIDE+1)
    v_ones = np.ones((1, CUBES_PER_SIDE+1))
    for i in values.ravel():
        for j in values.ravel():
            points = np.vstack([values, v_ones * i, v_ones * j, v_ones])
            traces.append(gen_points(points, transf_matrix))
            
            points = np.vstack([v_ones * i, values, v_ones * j, v_ones])
            traces.append(gen_points(points, transf_matrix))
            
            points = np.vstack([v_ones * i, v_ones * j, values, v_ones])
            traces.append(gen_points(points, transf_matrix))
            
    return traces

def wg84_axes():
    return axes(np.identity(4))

def subspace_axes(transf_matrix):
    return axes(transf_matrix)

def axes(transf_matrix):
    traces = []
    line_params = {'mode': 'lines', 'showlegend': False}
    points = transf_matrix @ np.array([[0,0,0,1],[AXES_LEN,0,0,1]]).T
    traces.append(go.Scatter3d(x=points[0,:], y=points[1,:], z=points[2,:], name='X', line=go.Line(color='#F00', width=3), **line_params))
    points = transf_matrix @ np.array([[0,0,0,1],[0,AXES_LEN,0,1]]).T
    traces.append(go.Scatter3d(x=points[0,:], y=points[1,:], z=points[2,:], name='Y', line=go.Line(color='#0F0', width=3), **line_params))
    points = transf_matrix @ np.array([[0,0,0,1],[0,0,AXES_LEN,1]]).T
    traces.append(go.Scatter3d(x=points[0,:], y=points[1,:], z=points[2,:], name='Z', line=go.Line(color='#00F', width=3), **line_params))
    return traces

def cities():
    traces = []
    mel = polar_to_cartesian(MELBOURNE_LAT, MELBOURNE_LON, RADIUS)
    syd = polar_to_cartesian(SYDNEY_LAT, SYDNEY_LON, RADIUS)
    c = np.vstack([mel, syd]).T
    traces.append(go.Scatter3d(x=c[0,:], y=c[1,:], z=c[2,:], mode='markers'))
    return traces

def polar_to_cartesian(lat, lon, alt):
    theta = lat - 90
    phi = lon
    x = alt * np.sin(theta) * np.cos(phi)
    y = alt * np.sin(theta) * np.sin(phi)
    z = alt * np.cos(theta)
    return np.array([x, y, z])

def sphere():
    step = RESOLUTION // LINES
    theta = np.linspace(0, 2 * np.pi, RESOLUTION)
    phi = np.linspace(0, np.pi, RESOLUTION)
    x = RADIUS * np.outer(np.cos(theta), np.sin(phi))
    y = RADIUS * np.outer(np.sin(theta), np.sin(phi))
    z = RADIUS * np.outer(np.ones(RESOLUTION), np.cos(phi))


    line_params = {'mode': 'lines', 'showlegend': False, 'line': go.Line(color='#CCC', width=2)}
    lon_lines, lat_lines = [], []
    for i in np.arange(0, x.shape[-1], step):
        lon_lines.append(go.Scatter3d(x=x[i,:], y=y[i,:], z=z[i,:], **line_params))
        lat_lines.append(go.Scatter3d(x=x[:,i], y=y[:,i], z=z[:,i], **line_params))
    
    return lon_lines + lat_lines

In [4]:
def plot_all(transf_matrix):
    data = go.Data(sphere() + wg84_axes() + subspace(transf_matrix) + subspace_axes(transf_matrix) + cities())
    layout = go.Layout(
        title='World and local spaces',
        autosize=False,
        width=1000,
        height=1000,
        margin=go.Margin(
            l=65,
            r=50,
            b=65,
            t=90
        )
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)


In [5]:
transf_matrix = np.identity(4)
plot_all(transf_matrix)

In [6]:
transf_matrix = np.identity(4)
SCALE = 1 / 2
transf_matrix[(0,1,2),(0,1,2)] = SCALE
M_scaling = transf_matrix

plot_all(M_scaling)

In [7]:
transf_matrix = np.identity(4)
mel = polar_to_cartesian(MELBOURNE_LAT, MELBOURNE_LON, RADIUS)
syd = polar_to_cartesian(SYDNEY_LAT, SYDNEY_LON, RADIUS)
middle = (mel + syd) / 2

# Base vector pointing outwards
e_3 = mel / norm(mel)
# Base vector parallel to the line connecting Melbourne to Sydney
e_1 = (mel - syd) / norm(mel - syd)
# Base vector perpendicular to the other two
e_2 = np.cross(e_3, e_1)

transf_matrix[(0,1,2),(0,0,0)] = e_1
transf_matrix[(0,1,2),(1,1,1)] = e_2
transf_matrix[(0,1,2),(2,2,2)] = e_3
M_rotation = transf_matrix

plot_all(M_scaling @ M_rotation)

In [8]:
transf_matrix = np.identity(4)
transf_matrix[(0,1,2),(3,3,3)] = [0,0,RADIUS/SCALE]
M_translation = transf_matrix

plot_all(M_scaling @ M_rotation @ M_translation)